In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import json
import folium
from pyproj import Proj, transform
import mapclassify as mc
from shapely.geometry import Polygon
import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs

from gurobipy import *
import gurobipy as gp

In [2]:
nn = pd.read_excel('neighbor_node.xlsx')
nn = nn.iloc[:,1:].fillna(0)
nn

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
82,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0


In [3]:
move_list = [(i,j) for i in range(84) for j in range(84) if nn.iloc[i,j]==1]
move_list

[(0, 0),
 (0, 1),
 (0, 4),
 (0, 5),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 5),
 (1, 6),
 (1, 7),
 (2, 1),
 (2, 2),
 (2, 6),
 (2, 7),
 (3, 3),
 (3, 4),
 (3, 8),
 (3, 9),
 (4, 0),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 8),
 (4, 9),
 (4, 10),
 (5, 0),
 (5, 1),
 (5, 4),
 (5, 5),
 (5, 6),
 (5, 9),
 (5, 10),
 (5, 11),
 (6, 0),
 (6, 1),
 (6, 2),
 (6, 5),
 (6, 6),
 (6, 7),
 (6, 10),
 (6, 11),
 (7, 1),
 (7, 2),
 (7, 6),
 (7, 7),
 (7, 11),
 (8, 3),
 (8, 4),
 (8, 8),
 (8, 9),
 (8, 13),
 (8, 14),
 (8, 15),
 (9, 3),
 (9, 4),
 (9, 5),
 (9, 8),
 (9, 9),
 (9, 10),
 (9, 14),
 (9, 15),
 (9, 16),
 (10, 4),
 (10, 5),
 (10, 6),
 (10, 9),
 (10, 10),
 (10, 11),
 (10, 15),
 (10, 16),
 (10, 17),
 (11, 5),
 (11, 6),
 (11, 7),
 (11, 10),
 (11, 11),
 (11, 16),
 (11, 17),
 (11, 18),
 (12, 12),
 (12, 13),
 (12, 20),
 (13, 8),
 (13, 12),
 (13, 13),
 (13, 14),
 (13, 20),
 (13, 21),
 (14, 8),
 (14, 9),
 (14, 13),
 (14, 14),
 (14, 15),
 (14, 20),
 (14, 21),
 (14, 22),
 (15, 8),
 (15, 9),
 (15, 10),
 (15, 14),
 (15, 15),
 (15, 16),
 

In [4]:
len(move_list)

631

In [5]:
conv_store = pd.read_excel('convenienceStore.xlsx')
conv_store

,no,상호명,구주소,신주소,위도,경도
0,1,세븐일레븐 반포서래마을점,서울특별시 서초구 반포동 612-55,"서울특별시 서초구 서래로5길 96, 1층 (반포동)",37.499411,126.998426
1,2,씨유 내방역점,서울특별시 서초구 방배동 897-1 1층,"서울특별시 서초구 서초대로26길 3, 1층 (방배동)",35.152398,126.869691
2,3,세븐일레븐 관악행운길점,서울특별시 관악구 봉천동 100-324,"서울특별시 관악구 행운6길 2, 1층 101호 (봉천동)",37.480639,126.960935
3,4,씨유 동대문도서관점,서울특별시 동대문구 신설동 104-30,"서울특별시 동대문구 천호대로2길 23-17, 1층 (신설동)",37.573624,127.025082
4,5,CU고덕래미안점,서울특별시 강동구 고덕동 688번지 래미안힐스테이트고덕,"서울특별시 강동구 아리수로50길 50, 1층 (고덕동, 래미안힐스테이트고덕)",37.558571,127.151149
...,...,...,...,...,...,...
946,947,씨유(CU) 동묘토성점,서울특별시 종로구 숭인동 318-1,"서울특별시 종로구 지봉로4길 21, 1층 101호 (숭인동)",37.571256,127.017102
947,948,GS25 논현우람점,서울특별시 강남구 논현동 159-11,"서울특별시 강남구 학동로6길 48, 지상1층 (논현동)",37.509233,127.026163
948,949,세븐일레븐 종로창의문점,서울특별시 종로구 부암동 220-1번지,서울특별시 종로구 창의문로 170 (부암동),37.594872,126.963856
949,950,씨유 성대원룸촌점,서울특별시 종로구 명륜1가 97-2번지,"서울특별시 종로구 성균관로 36, 1층 (명륜1가)",37.585882,126.997006


In [11]:
proj_2097 = Proj(init='EPSG:2097')
proj_4326 = Proj(init='EPSG:4326')

latlon_transform = transform(proj_4326, proj_2097, conv_store['경도'], conv_store['위도'])
latlon = pd.DataFrame(latlon_transform, index={'경도', '위도'}).transpose()
latlon['x_grid'] = (latlon['위도'] -180000)//3000
latlon['y_grid'] = (latlon['경도'] -435000)//3000
latlon['x_error'] = np.where((latlon['x_grid']>= 0) & (latlon['x_grid'] <= 13), "No", "Yes")
latlon['y_error'] = np.where((latlon['y_grid']>= 0) & (latlon['y_grid'] <= 11), "No", "Yes")
latlon['error'] = np.where((latlon['x_error'] == 'No') & (latlon['y_error'] == 'No'), "No", "Yes")
latlon['grid'] = latlon['x_grid']*12 + latlon['y_grid']
latlon

C:\Users\lotto\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\lotto\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\lotto\AppData\Local\Temp/ipykernel_29076/1644522192.py:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  latlon_transf

,위도,경도,x_grid,y_grid,x_error,y_error,error,grid
0,200046.628457,444133.527650,6.0,3.0,No,No,No,75.0
1,188315.967297,183702.707916,2.0,-84.0,No,Yes,Yes,-60.0
2,196730.632013,442050.662188,5.0,2.0,No,No,No,62.0
3,202401.427486,452370.629599,7.0,5.0,No,No,No,89.0
4,213540.842630,450710.477829,11.0,5.0,No,No,No,137.0
...,...,...,...,...,...,...,...,...
946,201696.525556,452107.617154,7.0,5.0,No,No,No,89.0
947,202499.032458,445223.972345,7.0,3.0,No,No,No,87.0
948,196993.648773,454729.170104,5.0,6.0,No,No,No,66.0
949,199921.287347,453730.804633,6.0,6.0,No,No,No,78.0


In [12]:
conv_store['index_before'] = latlon['grid']
conv_filtered = conv_store[latlon['error'] == 'No']
conv_filtered['index_before'] = conv_filtered['index_before'].astype(int)
grid3000 = pd.read_csv('grid3000.csv')
grid = grid3000[['index_before', 'index']]
conv_filtered = pd.merge(conv_filtered, grid, on='index_before', how='left')
conv_filtered = conv_filtered.dropna()
conv_filtered

C:\Users\lotto\AppData\Local\Temp/ipykernel_29076/3538499255.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conv_filtered['index_before'] = conv_filtered['index_before'].astype(int)


,no,상호명,구주소,신주소,위도,경도,index_before,index
0,1,세븐일레븐 반포서래마을점,서울특별시 서초구 반포동 612-55,"서울특별시 서초구 서래로5길 96, 1층 (반포동)",37.499411,126.998426,75,38.0
1,3,세븐일레븐 관악행운길점,서울특별시 관악구 봉천동 100-324,"서울특별시 관악구 행운6길 2, 1층 101호 (봉천동)",37.480639,126.960935,62,29.0
2,4,씨유 동대문도서관점,서울특별시 동대문구 신설동 104-30,"서울특별시 동대문구 천호대로2길 23-17, 1층 (신설동)",37.573624,127.025082,89,49.0
3,5,CU고덕래미안점,서울특별시 강동구 고덕동 688번지 래미안힐스테이트고덕,"서울특별시 강동구 아리수로50길 50, 1층 (고덕동, 래미안힐스테이트고덕)",37.558571,127.151149,137,82.0
4,6,세븐일레븐 한강광나루1호점,서울특별시 강동구 천호동 483-8,"서울특별시 강동구 한강남자전거길 3646, 1층 (천호동)",37.546207,127.120526,124,77.0
...,...,...,...,...,...,...,...,...
934,947,씨유(CU) 동묘토성점,서울특별시 종로구 숭인동 318-1,"서울특별시 종로구 지봉로4길 21, 1층 101호 (숭인동)",37.571256,127.017102,89,49.0
935,948,GS25 논현우람점,서울특별시 강남구 논현동 159-11,"서울특별시 강남구 학동로6길 48, 지상1층 (논현동)",37.509233,127.026163,87,47.0
936,949,세븐일레븐 종로창의문점,서울특별시 종로구 부암동 220-1번지,서울특별시 종로구 창의문로 170 (부암동),37.594872,126.963856,66,33.0
937,950,씨유 성대원룸촌점,서울특별시 종로구 명륜1가 97-2번지,"서울특별시 종로구 성균관로 36, 1층 (명륜1가)",37.585882,126.997006,78,41.0


In [13]:
conv_index = conv_filtered[['상호명', 'index']].reset_index(drop=True)
conv_index['index'] = conv_index['index'].astype(int)
conv_index

,상호명,index
0,세븐일레븐 반포서래마을점,38
1,세븐일레븐 관악행운길점,29
2,씨유 동대문도서관점,49
3,CU고덕래미안점,82
4,세븐일레븐 한강광나루1호점,77
...,...,...
925,씨유(CU) 동묘토성점,49
926,GS25 논현우람점,47
927,세븐일레븐 종로창의문점,33
928,씨유 성대원룸촌점,41


In [59]:
move_list.index((0,0))

0

In [14]:
conv_list = [1 if ((nn[i][conv_index['index'][k]]==1.0)|(nn[j][conv_index['index'][k]]==1.0)) else 0 for i,j in move_list for k in range(930)]
len(conv_list)

586830

In [15]:
y_list = [(i,j,k) for i,j in move_list for k in range(930) if conv_list[930*move_list.index((i,j))+k] == 1]
len(y_list)

85553

In [16]:
y_list

[(0, 0, 36),
 (0, 0, 47),
 (0, 0, 48),
 (0, 0, 127),
 (0, 0, 156),
 (0, 0, 187),
 (0, 0, 192),
 (0, 0, 196),
 (0, 0, 198),
 (0, 0, 199),
 (0, 0, 203),
 (0, 0, 528),
 (0, 0, 588),
 (0, 0, 603),
 (0, 0, 638),
 (0, 0, 657),
 (0, 0, 666),
 (0, 0, 695),
 (0, 0, 713),
 (0, 0, 789),
 (0, 0, 795),
 (0, 0, 804),
 (0, 0, 838),
 (0, 0, 851),
 (0, 0, 917),
 (0, 0, 919),
 (0, 1, 36),
 (0, 1, 47),
 (0, 1, 48),
 (0, 1, 127),
 (0, 1, 156),
 (0, 1, 187),
 (0, 1, 192),
 (0, 1, 196),
 (0, 1, 198),
 (0, 1, 199),
 (0, 1, 203),
 (0, 1, 528),
 (0, 1, 588),
 (0, 1, 603),
 (0, 1, 638),
 (0, 1, 657),
 (0, 1, 666),
 (0, 1, 695),
 (0, 1, 713),
 (0, 1, 789),
 (0, 1, 795),
 (0, 1, 804),
 (0, 1, 838),
 (0, 1, 851),
 (0, 1, 917),
 (0, 1, 919),
 (0, 4, 34),
 (0, 4, 36),
 (0, 4, 37),
 (0, 4, 47),
 (0, 4, 48),
 (0, 4, 65),
 (0, 4, 70),
 (0, 4, 84),
 (0, 4, 95),
 (0, 4, 107),
 (0, 4, 127),
 (0, 4, 156),
 (0, 4, 187),
 (0, 4, 189),
 (0, 4, 190),
 (0, 4, 191),
 (0, 4, 192),
 (0, 4, 193),
 (0, 4, 196),
 (0, 4, 198),
 (0, 4,

In [17]:
y_df = pd.DataFrame(y_list)
y_df[y_df[2] == 0].iloc[:,:-1].values.tolist()

[[20, 29],
 [21, 29],
 [21, 30],
 [22, 29],
 [22, 30],
 [22, 31],
 [23, 30],
 [23, 31],
 [24, 31],
 [28, 29],
 [28, 37],
 [29, 20],
 [29, 21],
 [29, 22],
 [29, 28],
 [29, 29],
 [29, 30],
 [29, 36],
 [29, 37],
 [29, 38],
 [30, 21],
 [30, 22],
 [30, 23],
 [30, 29],
 [30, 30],
 [30, 31],
 [30, 37],
 [30, 38],
 [30, 39],
 [31, 22],
 [31, 23],
 [31, 24],
 [31, 30],
 [31, 31],
 [31, 32],
 [31, 38],
 [31, 39],
 [31, 40],
 [32, 31],
 [32, 39],
 [36, 29],
 [36, 37],
 [36, 46],
 [37, 28],
 [37, 29],
 [37, 30],
 [37, 36],
 [37, 37],
 [37, 38],
 [37, 45],
 [37, 46],
 [37, 47],
 [38, 29],
 [38, 30],
 [38, 31],
 [38, 37],
 [38, 38],
 [38, 39],
 [38, 46],
 [38, 47],
 [38, 48],
 [39, 30],
 [39, 31],
 [39, 32],
 [39, 38],
 [39, 39],
 [39, 40],
 [39, 47],
 [39, 48],
 [39, 49],
 [40, 31],
 [40, 39],
 [40, 48],
 [45, 37],
 [45, 46],
 [46, 36],
 [46, 37],
 [46, 38],
 [46, 45],
 [46, 46],
 [46, 47],
 [46, 56],
 [46, 57],
 [46, 58],
 [47, 37],
 [47, 38],
 [47, 39],
 [47, 46],
 [47, 47],
 [47, 48],
 [47, 57],

In [18]:
motorcycle = grid3000['오토바이 등록대수']
traffic_list = [(motorcycle[i]+motorcycle[j])/2 for i,j in move_list]
len(traffic_list)

631

In [46]:
m = Model('LoadBalancing')
# m.ModelSense = GRB.MINIMIZE

m.setParam('TimeLimit', 60*60*2)

x = m.addVars(len(conv_index), vtype=GRB.BINARY, name="x")
y = m.addVars(y_list, vtype=GRB.BINARY, name="y")
L = m.addVar(vtype=GRB.CONTINUOUS, name="L_max")

m.setObjective(L, GRB.MINIMIZE)

m.addConstrs((y.sum(i,j,'*') == 1 for i,j in move_list), 'must_charger_visit')
m.addConstrs((y[i,j,k] <= x[k] for i,j,k in y_list), 'no_charger_no_visit')
for k in range(len(conv_index)):
    m.addConstr((gp.quicksum(traffic_list[move_list.index((i,j))]*y[i,j,k] for i,j in y_df[y_df[2] == k].iloc[:,:-1].values.tolist())) <= L )
# m.addConstrs((y.sum('*','*',k) <= L for k in range(len(conv_index))), 'L_max meaning')
# m.addConstrs((x.sum(k) <= 100 for k in range(len(conv_index))), 'charger_limit')
m.addConstr((gp.quicksum(x[k] for k in range(len(conv_index))) <= 100), 'charger_limit')

m.optimize()

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 87115 rows, 86484 columns and 344072 nonzeros
Model fingerprint: 0x8b30d161
Variable types: 1 continuous, 86483 integer (86483 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve time: 1.08s
Presolved: 87115 rows, 86484 columns, 344072 nonzeros
Variable types: 1 continuous, 86483 integer (86483 binary)
Found heuristic solution: objective 1.385490e+07
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 8

In [20]:
m.getVars()

[<gurobi.Var x[0] (value 1.0)>,
 <gurobi.Var x[1] (value -0.0)>,
 <gurobi.Var x[2] (value 1.0)>,
 <gurobi.Var x[3] (value -0.0)>,
 <gurobi.Var x[4] (value -0.0)>,
 <gurobi.Var x[5] (value -0.0)>,
 <gurobi.Var x[6] (value -0.0)>,
 <gurobi.Var x[7] (value -0.0)>,
 <gurobi.Var x[8] (value -0.0)>,
 <gurobi.Var x[9] (value -0.0)>,
 <gurobi.Var x[10] (value -0.0)>,
 <gurobi.Var x[11] (value -0.0)>,
 <gurobi.Var x[12] (value -0.0)>,
 <gurobi.Var x[13] (value -0.0)>,
 <gurobi.Var x[14] (value -0.0)>,
 <gurobi.Var x[15] (value 0.0)>,
 <gurobi.Var x[16] (value -0.0)>,
 <gurobi.Var x[17] (value -0.0)>,
 <gurobi.Var x[18] (value -0.0)>,
 <gurobi.Var x[19] (value 1.0)>,
 <gurobi.Var x[20] (value -0.0)>,
 <gurobi.Var x[21] (value 0.0)>,
 <gurobi.Var x[22] (value -0.0)>,
 <gurobi.Var x[23] (value -0.0)>,
 <gurobi.Var x[24] (value 0.0)>,
 <gurobi.Var x[25] (value -0.0)>,
 <gurobi.Var x[26] (value -0.0)>,
 <gurobi.Var x[27] (value 1.0)>,
 <gurobi.Var x[28] (value -0.0)>,
 <gurobi.Var x[29] (value -0.0)

In [48]:
m.getVars()

[<gurobi.Var x[0] (value -0.0)>,
 <gurobi.Var x[1] (value -0.0)>,
 <gurobi.Var x[2] (value -0.0)>,
 <gurobi.Var x[3] (value -0.0)>,
 <gurobi.Var x[4] (value -0.0)>,
 <gurobi.Var x[5] (value -0.0)>,
 <gurobi.Var x[6] (value -0.0)>,
 <gurobi.Var x[7] (value -0.0)>,
 <gurobi.Var x[8] (value -0.0)>,
 <gurobi.Var x[9] (value -0.0)>,
 <gurobi.Var x[10] (value -0.0)>,
 <gurobi.Var x[11] (value 1.0)>,
 <gurobi.Var x[12] (value -0.0)>,
 <gurobi.Var x[13] (value -0.0)>,
 <gurobi.Var x[14] (value -0.0)>,
 <gurobi.Var x[15] (value -0.0)>,
 <gurobi.Var x[16] (value -0.0)>,
 <gurobi.Var x[17] (value -0.0)>,
 <gurobi.Var x[18] (value 1.0)>,
 <gurobi.Var x[19] (value -0.0)>,
 <gurobi.Var x[20] (value 1.0)>,
 <gurobi.Var x[21] (value -0.0)>,
 <gurobi.Var x[22] (value -0.0)>,
 <gurobi.Var x[23] (value -0.0)>,
 <gurobi.Var x[24] (value -0.0)>,
 <gurobi.Var x[25] (value 1.0)>,
 <gurobi.Var x[26] (value -0.0)>,
 <gurobi.Var x[27] (value -0.0)>,
 <gurobi.Var x[28] (value -0.0)>,
 <gurobi.Var x[29] (value -0

In [24]:
m.objVal

771609.3902885652

In [47]:
m.objVal

726202.5580430108

In [38]:
len(m.getAttr('x', m.getVars()))

86484

In [51]:
assignment=[v.X for v in m.getVars() if v.VarName[0]=="x"]
assignment

[-0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 1.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0

In [53]:
conv_fin_model = conv_index
conv_fin_model['construct'] = assignment
conv_fin_model[conv_fin_model['construct'] == 1]

,상호명,index,construct
11,세븐일레븐 디지털타워점,59,1.0
18,이마트24 장안라보니타점,60,1.0
20,세븐일레븐 가산캐슬점,14,1.0
25,씨유성내중앙점,76,1.0
36,GS공항소담점,6,1.0
...,...,...,...
878,씨유 강남거평점,47,1.0
887,세븐일레븐 명륜카페점,41,1.0
900,씨유 가산승리점,14,1.0
924,씨유(CU) 동대문역점,49,1.0


In [71]:
restaurants = pd.read_csv('restaurants.csv', encoding='cp949')
restaurants

x_tm = [180000 + 3000*x for x in range(0, 14)]
y_tm = [435000 + 3000*x for x in range(0, 12)]
x_grid = [x//12 for x in range(0, 168)]
y_grid = [x%12 for x in range(0,168)]
grid = pd.DataFrame([x_grid, y_grid], index=['x_grid', 'y_grid']).transpose()
grid['x_tm_start'] = [x_tm[grid.iloc[x, 0]] for x in range(0,168)]
grid['x_tm_end'] = grid['x_tm_start'] + 3000
grid['y_tm_start'] = [y_tm[grid.iloc[x, 1]] for x in range(0,168)]
grid['y_tm_end'] = grid['y_tm_start'] + 3000

restaurants_necessary = restaurants[['좌표정보(X)', '좌표정보(Y)']]
restaurants_necessary = restaurants_necessary.dropna()
restaurants_necessary['x_grid'] = (restaurants_necessary['좌표정보(X)'] -180000)//3000
restaurants_necessary['y_grid'] = (restaurants_necessary['좌표정보(Y)'] -435000)//3000
restaurants_necessary['restaurants'] = restaurants_necessary['x_grid']*12 + restaurants_necessary['y_grid']
grid_index = pd.DataFrame(restaurants_necessary['restaurants'].astype(int).value_counts())
grid = pd.merge(grid, grid_index, left_index=True, right_index=True, how='left')
grid = grid.fillna(0)

proj_2097 = Proj(init='EPSG:2097')
proj_4326 = Proj(init='EPSG:4326')

start_transform = transform(proj_2097, proj_4326, grid['x_tm_start'], grid['y_tm_start'])
end_transform = transform(proj_2097, proj_4326, grid['x_tm_end'], grid['y_tm_end'])
grid_all = pd.merge(grid, pd.DataFrame(start_transform, index=['lat_start', 'lon_start']).transpose(), left_index=True, right_index=True, how='left')
grid_all = pd.merge(grid_all, pd.DataFrame(end_transform, index=['lat_end', 'lon_end']).transpose(), left_index=True, right_index=True, how='left')

grid_list = [[[grid_all.iloc[x, 8], grid_all.iloc[x, 7]], 
             [grid_all.iloc[x, 10], grid_all.iloc[x, 7]], 
             [grid_all.iloc[x, 10], grid_all.iloc[x, 9]],
             [grid_all.iloc[x, 8], grid_all.iloc[x, 9]],
             [grid_all.iloc[x, 8], grid_all.iloc[x, 7]]] for x in range(0,168)]

polygon = []

for grid in grid_list:
    polygon.append(Polygon(grid))

grid3000 = gpd.GeoDataFrame({'geometry':polygon})
grid3000.crs = 'EPSG:4326'
grid3000['index'] = grid3000.index
grid3000['restaurants'] = grid_all['restaurants'].astype(int)
grid3000 = grid3000[grid3000['restaurants'] != 0].reset_index(drop=True)

grid_list = [[[grid_all.iloc[x, 7], grid_all.iloc[x, 8]], 
             [grid_all.iloc[x, 7], grid_all.iloc[x, 10]], 
             [grid_all.iloc[x, 9], grid_all.iloc[x, 10]],
             [grid_all.iloc[x, 9], grid_all.iloc[x, 8]],
             [grid_all.iloc[x, 7], grid_all.iloc[x, 8]]] for x in range(0,168)]

polygon = []

for grid in grid_list:
    polygon.append(Polygon(grid))

grid3000 = gpd.GeoDataFrame({'geometry':polygon})
grid3000.crs = 'EPSG:4326'
grid3000['index_before'] = grid3000.index
grid3000['restaurants'] = grid_all['restaurants'].astype(int)
grid3000 = grid3000[grid3000['restaurants'] != 0].reset_index(drop=True)
grid3000['index'] = grid3000.index

grid3000['station'] = 0
for i in range(len(conv_fin_model)):
    grid3000['station'][conv_fin_model['index'][i]] += 1
grid3000

C:\Users\lotto\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (25,27,28,29,30,36,39,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\lotto\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\lotto\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _pr

,geometry,index_before,restaurants,index,station
0,"POLYGON ((126.77165 37.52502, 126.77165 37.552...",4,5,0,0
1,"POLYGON ((126.77156 37.55205, 126.77156 37.579...",5,151,1,0
2,"POLYGON ((126.77148 37.57908, 126.77148 37.606...",6,2,2,0
3,"POLYGON ((126.80572 37.47102, 126.80572 37.498...",14,300,3,4
4,"POLYGON ((126.80565 37.49805, 126.80565 37.525...",15,367,4,5
...,...,...,...,...,...
79,"POLYGON ((127.14486 37.47109, 127.14486 37.498...",134,237,79,1
80,"POLYGON ((127.14491 37.49812, 127.14491 37.525...",135,73,80,0
81,"POLYGON ((127.14496 37.52515, 127.14496 37.552...",136,231,81,27
82,"POLYGON ((127.14502 37.55218, 127.14502 37.579...",137,339,82,37


In [74]:
grid3000['station'] = 0

conv_true = conv_fin_model[conv_fin_model['construct'] == 1].reset_index(drop=True)

for i in range(len(conv_true)):
    grid3000['station'][conv_true['index'][i]] += 1
grid3000

C:\Users\lotto\AppData\Local\Temp/ipykernel_29076/4125394111.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grid3000['station'][conv_true['index'][i]] += 1


,geometry,index_before,restaurants,index,station
0,"POLYGON ((126.77165 37.52502, 126.77165 37.552...",4,5,0,0
1,"POLYGON ((126.77156 37.55205, 126.77156 37.579...",5,151,1,0
2,"POLYGON ((126.77148 37.57908, 126.77148 37.606...",6,2,2,0
3,"POLYGON ((126.80572 37.47102, 126.80572 37.498...",14,300,3,0
4,"POLYGON ((126.80565 37.49805, 126.80565 37.525...",15,367,4,1
...,...,...,...,...,...
79,"POLYGON ((127.14486 37.47109, 127.14486 37.498...",134,237,79,0
80,"POLYGON ((127.14491 37.49812, 127.14491 37.525...",135,73,80,0
81,"POLYGON ((127.14496 37.52515, 127.14496 37.552...",136,231,81,1
82,"POLYGON ((127.14502 37.55218, 127.14502 37.579...",137,339,82,0


In [75]:
m = folium.Map(location=[37.5642135, 127.0016985], zoom_start=11)

sim_geo = gpd.GeoSeries(grid3000['geometry'], grid3000['index'])
geo_j = sim_geo.to_json()

folium.Choropleth(
    geo_data=geo_j,
    data=grid3000,
    columns=['index', 'station'],
    key_on='id',
    fill_color='YlGnBu'
).add_to(m)



for _, r in grid3000.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo_new = gpd.GeoSeries(r['geometry'])
    geo_j_new = sim_geo_new.to_json()
    geo_j_new = folium.GeoJson(data=geo_j_new, style_function=lambda x:{'fillColor': '#FF000000', 'color': '#FF000000'})
    folium.Popup(str(r['station'])).add_to(geo_j_new)
    geo_j_new.add_to(m)

m

In [80]:
conv_filtered.sort_values(by='index')

,no,상호명,구주소,신주소,위도,경도,index_before,index
74,78,GS25항동수목원점,서울특별시 구로구 항동 1 동삼파크빌라,"서울특별시 구로구 연동로 312, 동삼파크빌라 상가9동 104,105호 (항동)",37.487748,126.824396,14,3.0
99,103,씨유 항동3단지점,서울특별시 구로구 항동 128-2 가온 프라자,"서울특별시 구로구 항동로3길 6, 가온 프라자 1층 107,108호 (항동)",37.480800,126.824325,14,3.0
615,624,세븐일레븐 항동하버라인점,서울특별시 구로구 항동 22-10,"서울특별시 구로구 연동로11길 26, 1층 (항동)",37.483027,126.821508,14,3.0
554,561,지에스25 구로철길점,서울특별시 구로구 오류동 358 다원리치타운,"서울특별시 구로구 오리로11길 99, 상가동 1층 101호 (오류동, 다원리치타운)",37.487138,126.831167,14,3.0
192,197,세븐일레븐신월지양로점,서울특별시 양천구 신월동 953-5 지상1층,"서울특별시 양천구 지양로 63, 지상1층 (신월동)",37.521204,126.834078,15,4.0
...,...,...,...,...,...,...,...,...
303,308,씨유 강일풍성점,서울특별시 강동구 강일동 686 강일리버파크7단지아파트,"서울특별시 강동구 아리수로98길 25, 상가동 1층 104,105호 (강일동, 강일...",37.564806,127.178554,137,82.0
325,330,지에스25 강일MK점,서울특별시 강동구 강일동 76-7 MK프라자,"서울특별시 강동구 아리수로93나길 50, MK프라자 1층 108호 (강일동)",37.572101,127.174498,137,82.0
309,314,씨유 고덕자이점,서울특별시 강동구 상일동 124,"서울특별시 강동구 상일로 55, 고덕자이아파트 상가동 115호 (상일동)",37.553318,127.168786,137,82.0
396,401,씨유 리엔파크본점,서울특별시 강동구 상일동 36-3 강동리엔파크11단지,"서울특별시 강동구 고덕로98길 101, 상가동 제지1층 103호 (상일동, 강동리엔...",37.554246,127.180294,149,83.0


In [81]:
dna_motors = pd.read_csv('dna_motors.csv', encoding='euc_kr')
dna_motors

,name,주소,위도,경도
0,방배먹자골목 입구,서울 서초구 방배로 87,37.482111,126.996944
1,신당동떡볶이 입구,서울 중구 신당동 292-8?,37.562622,127.014974
2,건설회관 건너,서울 강남구 논현동 114,37.516889,127.035861
3,가락2동주민센터 뒤,서울 송파구 가락동 154?,37.498670,127.126590
4,싸리재공원,서울 서초구 잠원동 30-10,37.513368,127.018245
...,...,...,...,...
143,DNA서부기동점,서울 영등포구 신길동 215-5,37.511269,126.912167
144,모자인,서울 마포구 창전동 15-9,37.549769,126.931869
145,DNA종합모터스,서울 동대문구 답십리동 1-45,37.574168,127.066834
146,CU신림카페점,서울 관악구 신림로 228,37.477704,126.936146


In [94]:
dist=[]
for i in conv_filtered.index:
    minimum = 100
    for j in dna_motors.index:
        if (abs(conv_filtered['위도'][i]-dna_motors['위도'][j])+abs(conv_filtered['경도'][i]-dna_motors['경도'][j])) < minimum:
            minimum = (abs(conv_filtered['위도'][i]-dna_motors['위도'][j])+abs(conv_filtered['경도'][i]-dna_motors['경도'][j]))
    dist.append(minimum)
conv_filtered['minimum_dist'] = dist
conv_filtered

,no,상호명,구주소,신주소,위도,경도,index_before,index,minimum_dist
0,1,세븐일레븐 반포서래마을점,서울특별시 서초구 반포동 612-55,"서울특별시 서초구 서래로5길 96, 1층 (반포동)",37.499411,126.998426,75,38.0,0.002096
1,3,세븐일레븐 관악행운길점,서울특별시 관악구 봉천동 100-324,"서울특별시 관악구 행운6길 2, 1층 101호 (봉천동)",37.480639,126.960935,62,29.0,0.008209
2,4,씨유 동대문도서관점,서울특별시 동대문구 신설동 104-30,"서울특별시 동대문구 천호대로2길 23-17, 1층 (신설동)",37.573624,127.025082,89,49.0,0.007477
3,5,CU고덕래미안점,서울특별시 강동구 고덕동 688번지 래미안힐스테이트고덕,"서울특별시 강동구 아리수로50길 50, 1층 (고덕동, 래미안힐스테이트고덕)",37.558571,127.151149,137,82.0,0.038219
4,6,세븐일레븐 한강광나루1호점,서울특별시 강동구 천호동 483-8,"서울특별시 강동구 한강남자전거길 3646, 1층 (천호동)",37.546207,127.120526,124,77.0,0.005642
...,...,...,...,...,...,...,...,...,...
934,947,씨유(CU) 동묘토성점,서울특별시 종로구 숭인동 318-1,"서울특별시 종로구 지봉로4길 21, 1층 101호 (숭인동)",37.571256,127.017102,89,49.0,0.002871
935,948,GS25 논현우람점,서울특별시 강남구 논현동 159-11,"서울특별시 강남구 학동로6길 48, 지상1층 (논현동)",37.509233,127.026163,87,47.0,0.006790
936,949,세븐일레븐 종로창의문점,서울특별시 종로구 부암동 220-1번지,서울특별시 종로구 창의문로 170 (부암동),37.594872,126.963856,66,33.0,0.018571
937,950,씨유 성대원룸촌점,서울특별시 종로구 명륜1가 97-2번지,"서울특별시 종로구 성균관로 36, 1층 (명륜1가)",37.585882,126.997006,78,41.0,0.011225


In [113]:
conv_for_rank = conv_filtered
conv_for_rank['rank'] = conv_for_rank.groupby('index')['minimum_dist'].rank(method='min', ascending=False)
conv_for_rank = conv_for_rank.sort_values(by=['index', 'rank']).reset_index(drop=True)
conv_for_rank = pd.merge(conv_for_rank, grid3000[['index', 'station']], on='index', how='left')
conv_for_rank['construct'] = np.where(conv_for_rank['rank'] <= conv_for_rank['station'], 1, 0)
conv_finish = conv_for_rank[['no', '상호명', '구주소', '신주소', 'construct']].sort_values(by='no').reset_index(drop=True)
conv_finish

,no,상호명,구주소,신주소,construct
0,1,세븐일레븐 반포서래마을점,서울특별시 서초구 반포동 612-55,"서울특별시 서초구 서래로5길 96, 1층 (반포동)",0
1,3,세븐일레븐 관악행운길점,서울특별시 관악구 봉천동 100-324,"서울특별시 관악구 행운6길 2, 1층 101호 (봉천동)",0
2,4,씨유 동대문도서관점,서울특별시 동대문구 신설동 104-30,"서울특별시 동대문구 천호대로2길 23-17, 1층 (신설동)",0
3,5,CU고덕래미안점,서울특별시 강동구 고덕동 688번지 래미안힐스테이트고덕,"서울특별시 강동구 아리수로50길 50, 1층 (고덕동, 래미안힐스테이트고덕)",0
4,6,세븐일레븐 한강광나루1호점,서울특별시 강동구 천호동 483-8,"서울특별시 강동구 한강남자전거길 3646, 1층 (천호동)",0
...,...,...,...,...,...
925,947,씨유(CU) 동묘토성점,서울특별시 종로구 숭인동 318-1,"서울특별시 종로구 지봉로4길 21, 1층 101호 (숭인동)",0
926,948,GS25 논현우람점,서울특별시 강남구 논현동 159-11,"서울특별시 강남구 학동로6길 48, 지상1층 (논현동)",0
927,949,세븐일레븐 종로창의문점,서울특별시 종로구 부암동 220-1번지,서울특별시 종로구 창의문로 170 (부암동),1
928,950,씨유 성대원룸촌점,서울특별시 종로구 명륜1가 97-2번지,"서울특별시 종로구 성균관로 36, 1층 (명륜1가)",0


In [117]:
conv_finish.to_excel('후보지 충전소 건설 여부.xlsx')